In [31]:
import os
import numpy as np
import pandas as pd

import plotly.express as px

from tqdm.auto import tqdm
from minisom import MiniSom

from sklearn.preprocessing import StandardScaler

In [32]:
folder = os.path.join(
    os.environ["SYNC"], r"MotionData\data\2nd-Development", "parsedData"
)

event_name = {
    "0-SLW": 1,
    "1-MLW": 2,
    "2-FLW": 3,
    "3-RD": 4,
    "4-SD": 5,
    "5-sit": 6,
    "6-stand": 7,
    "7-RA": 8,
    "8-SA": 9,
}

In [33]:
X3s = []
ys = []

# Read and mark the Signal
for name, label in event_name.items():
    with open(os.path.join(folder, f'{name}.npy'), 'rb') as f:
        mat = np.load(f)
    X3s.append(mat)
    ys.append(np.zeros(mat.shape[0]) + label)

X3 = np.concatenate(X3s)
y = np.concatenate(ys)

# Flatten and Preprocess the Signal
X = X3.copy().reshape(-1, 120)

# for j in tqdm(range(120)):
#     a = X[:, j]
#     b = np.max(a)
#     c = np.min(a)
#     a -= c
#     if b > c:
#         a /= b-c

# Simple Report
X3.shape, X.shape, y.shape

((31625, 20, 6), (31625, 120), (31625,))

In [4]:
if os.path.isfile(os.path.join(folder, "positions.npy")):
    with open(os.path.join(folder, "positions.npy"), "rb") as f:
        y = np.load(f)
        origins = np.load(f)
        finishs = np.load(f)
else:
    origins = []
    finishs = []

    for X in tqdm(X3):
        mt = MotionTrace()

        for d in X:
            mt.forward(ap=d[3:], aax=d[0], aay=d[1], aaz=d[2])
        origins.append(mt.trace_origin()[np.newaxis, :])
        finishs.append(mt.trace_finish()[np.newaxis, :])

    origins = np.array(origins)
    finishs = np.array(finishs)

    with open(os.path.join(folder, "positions.npy"), "wb") as f:
        np.save(f, y)
        np.save(f, origins)
        np.save(f, finishs)

origins.shape, finishs.shape

((31625, 1, 21, 3), (31625, 1, 21, 3))

In [5]:
origins = origins.squeeze()
finishs = finishs.squeeze()
[print(e.shape) for e in [origins, finishs, y]]

(31625, 21, 3)
(31625, 21, 3)
(31625,)


[None, None, None]

In [11]:
X = finishs
X.shape, y.shape

((31625, 21, 3), (31625,))

In [14]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10)

for train, test in skf.split(y, y):
    break
    
test.shape


(3163,)

In [29]:
df = pd.DataFrame(X[test][:, -1, :].squeeze(), columns=['x', 'y', 'z'])
df['label'] = [str(e) for e in y[test]]
df['size'] = 1
df

,x,y,z,label,size
0,6281.247567,1849.594926,8124.708907,1.0,1
1,-6577.008932,4196.711196,-1882.663869,1.0,1
2,-21969.698508,4374.534582,275.547883,1.0,1
3,-788.964481,-1094.285039,2922.480118,1.0,1
4,29256.796536,-6295.500600,-4345.873514,1.0,1
...,...,...,...,...,...
3158,7856.245079,-2912.227472,6933.654511,9.0,1
3159,-17911.351718,-8346.725448,-20149.818912,9.0,1
3160,-4167.031315,7645.663091,-10697.244116,9.0,1
3161,-3636.090453,-626.303169,-18821.167473,9.0,1


In [30]:
title = 'Finishes'
fig = px.scatter_3d(
    df,
    x='x',
    y='y',
    z='z',
    color='label',
    size='size',
    size_max=10,
    height=1000,
    width=1000,
    title=title,
)
fig.show()

In [6]:
X1 = origins.reshape((len(origins), -1))
X2 = finishs.reshape((len(finishs), -1))
X = np.concatenate([X1, X2], axis=1)
print(X.shape) 

(31625, 126)


In [7]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X[0]

array([ 0.        ,  0.        ,  0.        , -0.7220919 ,  1.7651605 ,
        1.03270258, -0.75325157,  1.74499644,  1.10473805, -0.63027434,
        1.75438882,  1.15784408, -0.39095185,  1.77175776,  1.16738289,
       -0.09715916,  1.78661858,  1.14677397,  0.21228118,  1.79309931,
        1.10458781,  0.50472134,  1.78867257,  1.06091921,  0.75281622,
        1.77238591,  1.03338238,  0.94321167,  1.74753091,  1.03118383,
        1.0769529 ,  1.71878398,  1.05865326,  1.16448364,  1.68675163,
        1.11495687,  1.21912566,  1.64469936,  1.19736383,  1.25038409,
        1.57346192,  1.29974758,  1.26240432,  1.44741148,  1.40739284,
        1.25398912,  1.24841637,  1.49339011,  1.21796186,  0.99259085,
        1.5213431 ,  1.14811484,  0.73062475,  1.46886757,  1.04331768,
        0.51027986,  1.35619154,  0.90699991,  0.35609333,  1.22759425,
        0.73994207,  0.28921978,  1.11345723,  0.        ,  0.        ,
        0.        , -0.76795116,  1.76510146,  1.06740799, -0.80

In [36]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [39]:
# compute som size by counting samples
n = np.ceil(np.sqrt(5 * np.sqrt(X.shape[0]))).astype(int)

# Initialize som using counting of features
som = MiniSom(n, n, X.shape[1], sigma=5.0, learning_rate=0.05)

# trains the SOM with 100 iterations
som.train(X, 10000, verbose=True)

print(f'SOM ({n}, {n}, {X.shape[1]}) is trained.')

 [   366 / 10000 ]   4% - 0:00:04 left 

 [   742 / 10000 ]   7% - 0:00:04 left 

 [  1127 / 10000 ]  11% - 0:00:04 left 

 [  1515 / 10000 ]  15% - 0:00:04 left 

 [  1912 / 10000 ]  19% - 0:00:03 left 

...

 [  8958 / 10000 ]  90% - 0:00:00 left 

 [  9350 / 10000 ]  94% - 0:00:00 left 

 [  9739 / 10000 ]  97% - 0:00:00 left 

 [ 10000 / 10000 ] 100% - 0:00:00 left 


 quantization error: 6.618053102677936
SOM (30, 30, 120) is trained.


In [40]:
for e in np.unique(y):
    t = np.mean(X[y==e], axis=0)
    fig = px.scatter(t)
    fig.show() 
    act = som.activate(t)
    fig = px.imshow(act, title=f'{e} {len(y[y==e])}')
    fig.show()